![Curso Schwarz-Sosa-Suriano](http://www.fi.uba.ar/sites/default/files/logo.png)
# Sistemas de Ecuaciones Lineales
## Métodos Directos
***
**Curso Schwarz - Sosa - Suriano**
- Métodos Numéricos. *Curso 2*
- Análisis Numérico I. *Curso 4*
- Métodos Matemáticos y Numéricos. *Curso 6*

### Factorización de Matrices:

Resolver un SEL cualquiera, sin que la matriz A sea particularmente rala, mediante eliminación Gaussiana requiere una cantidad de operaciones $O({n^3}/3)$ siendo $n$ la cantidad de ecuaciones del sistema.

Si por el contrario tenemos una matriz A que sea diagonal inferior o diagonal superior,  se requiere una cantidad de operaciones proporcional a $O(n^2)$. 

Una forma de aprovechar esta reducción sustancial en la cantidad de opraciones necesarias es factorizando $A$ en la forma $A = L \cdot U$, donde $L$ es diagonal inferior y $U$ es diagonal superior. De esta manera:
1. Resolvemos primero $L \cdot Y = B $ 
2. Y finalmente $U \cdot X = Y $

Con esto conseguiremos reducir la cantidad de operaciones de $O({n^3}/3)$ a $O(2.{n^2})$ para resolver el mismo SEL.

Ahora bien, ¿cuál es el costo computacional de la factorización?

Lamentablemente, la cantidad de operaciones necesarias es también $O({n^3}/3)$.

No obstante, cada vez que debamos recalcular un SEL que comparta la misma matriz A con nuestro sistema original, requeriremos solamente  $O(2.{n^2})$ operaciones, por lo que el esfuerzo de la factorización será siempre justificado.

Antes de avanzar con los métodos de factorización, vamos a presentar el problema con el que trabajaremos, definiendo para ello la matriz A y el vector B:

In [1]:
import numpy as np
A = np.matrix([[15,  9,  9, 11], [ 9, 12,  7,  8], [ 9,  7,  7,  7], [11,  8,  7, 10]])
print(A)
B = np.array([1,2,3,3])

[[15  9  9 11]
 [ 9 12  7  8]
 [ 9  7  7  7]
 [11  8  7 10]]


In [2]:
import numpy as np
B = np.array([1,2,3])
A = np.matrix([[15,  9,  9], [ 9, 12,  7], [ 9,  7,  7]])
print(A)

[[15  9  9]
 [ 9 12  7]
 [ 9  7  7]]


### Factorización de Doolittle:

$$L_{i,i} . U_{i,i} = A_{i,i} - \sum_{k=1}^{i-1}{L_{i,k}.U_{k,i}} \hspace{5mm} con \hspace{5mm} L_{i,i} = 1 \implies
U_{i,i} = A_{i,i} - \sum_{k=1}^{i-1}{L_{i,k}.U_{k,i}}$$

$$U_{i,j} = \frac{A_{i,j} - \sum_{k=1}^{i-1}{L_{i,k}.U_{k,j}}}{L_{i,i}} \hspace{10mm} L_{j,i} = \frac{A_{j,i} - \sum_{k=1}^{i-1}{L_{j,k}.U_{k,i}}}{U_{i,i}}  \hspace{5mm} con \hspace{5mm} j \in [i+1,n] $$

En el bloque de código encontrarán una función `factorizarDoolittle` que permite realizar la factorización , y otra `bmatrix` que nos permite presentar una matriz en forma un poco más estética en formato $\LaTeX$.

Para factorizar una matriz A bastará con hacer `factorizarDoolittle(A)`, y además de obtener L y U como parámetros de salida, obtendrán una visualización en $\LaTeX$ paso a paso. 

Si solamente quieren factorizar sin que aparezca la resolución paso a paso, basta con hacer `factorizarDoolittle(A, False)`

Y si quieren obtener un código más limpio, sin ninguna referencia de paso a paso, pueden eliminar cada uno de los bloques que comienzan con `if (pasos):`

In [3]:
from IPython.display import display, Math

def factorizarDoolittle(A,pasos=True):
    n = np.shape(A)[0]
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    for i in range (0,n):
        L[i,i] = 1
        U[i,i] = A[i,i]  
        if (pasos):
            display(Math("\\text{Vamos a calcular la "+str(i+1)+"ª fila de U y la "+str(i+1)+"ª columna de L}"))
            display(Math("\\text{Por definición } L_{"+str(i+1)+","+str(i+1)+"} = 1"))                      
            display(Math("\\text{Y para U:}"))    
            exp = "U_{"+str(i+1)+","+str(i+1)+"} = A_{"+str(i+1)+","+str(i+1)+"}"
            expN = "U_{"+str(i+1)+","+str(i+1)+"} = "+str(A[i,i])        
        for k in range(0,i):
            U[i,i] -= L[i,k]*U[k,i] 
            if (pasos):
                exp += " - L_{"+str(i+1)+","+str(k+1)+"} \cdot U_{"+str(k+1)+","+str(i+1)+"}"
                expN += " - "+str(L[i,k])+"\cdot"+str(U[k,i])
        if (pasos):
            expN += "="+str(U[i,i])            
            if (i>2):
                display(Math(exp))
                display(Math(expN))
            else:
                display(Math(exp+"\\implies "+expN))
            display(Math("\\text{Seguimos con la "+str(i+1)+"ª fila de U y la "+str(i+1)+"ª columna de L }"))            
        for j in range (i+1,n):         
            U[i,j] = A[i,j]             
            L[j,i] = A[j,i]
            if (pasos):
                exp1 = "U_{"+str(i+1)+","+str(j+1)+"} = \\frac{A_{"+str(i+1)+","+str(j+1)+"}"  
                exp1N = "U_{"+str(i+1)+","+str(j+1)+"} = \\frac{"+str(A[i,j])
                exp2 = "L_{"+str(j+1)+","+str(i+1)+"} = \\frac{A_{"+str(j+1)+","+str(i+1)+"}" 
                exp2N = "L_{"+str(j+1)+","+str(i+1)+"} = \\frac{"+str(A[j,i])                
            for k in range(0,i):           
                U[i,j] -= L[i,k]*U[k,j]                  
                L[j,i] -= L[j,k]*U[k,i] 
                if (pasos):
                    exp1  += " - L_{"+str(i+1)+","+str(k+1)+"} \cdot U_{"+str(k+1)+","+str(j+1)+"}"  
                    exp1N += " - "+str(L[i,k])+" \cdot "+str(U[k,j])                     
                    exp2  += " - L_{"+str(j+1)+","+str(k+1)+"} \cdot U_{"+str(k+1)+","+str(i+1)+"}" 
                    exp2N += " - "+str(L[j,k])+" \cdot "+str(U[k,i])                     
            U[i,j] = U[i,j]/L[i,i] 
            L[j,i] = L[j,i]/U[i,i]
            if (pasos):
                exp1  += "}{L_{"+str(i+1)+","+str(i+1)+"}}"   
                exp1N += "}{1} = "+str(U[i,j])               
                exp2  += "}{U_{"+str(i+1)+","+str(i+1)+"}}"  
                exp2N += "}{"+str(U[i,i])+"} = "+str(L[j,i])             
                if (i>2):
                    display(Math(exp1))
                    display(Math(exp1N))                
                    display(Math(exp2)) 
                    display(Math(exp2N))   
                else:
                    display(Math(exp1+"\\implies "+exp1N))
                    display(Math(exp2+"\\implies "+exp2N))
        if (pasos):
            if (i<n-1):
                display(Math("\\text{Y hasta acá tenemos:}"))                
            else:
                display(Math("\\text{Y ahora sí tenemos L y U completas:}"))
            display(Math("L = "+bmatrix(L)+" \\hspace{10mm} U = "+bmatrix(U)+" \\hspace{10mm} A = "+bmatrix(A)))
    return L,U 

def bmatrix(a):
    # credito: https://stackoverflow.com/questions/17129290/numpy-2d-and-1d-array-to-latex-bmatrix
    if len(a.shape) > 2:
        raise ValueError('solo vectores o matrices bidimensionales')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [4]:
L, U = factorizarDoolittle(A)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Sustitución Directa:

Utilizaremos la sustitución directa para hallar el vector $Y$ que resuelve nuestro SEL mediante la resolución del sistema $ L \cdot Y=B$. Para ello, tenemos el siguiente código, al que se le puede aplicar un tercer parámetro `False` para ahorrar la salida paso a paso:

In [5]:
def sustDirecta(L, B, pasos = True):
    n = np.shape(L)[0]
    Y = np.zeros(n)    
    if (pasos):
        display(Math("\\text{Tenemos:}"))                
        exp = bmatrix(L)+" \cdot \\begin{bmatrix}"
        for i in range(0,n): exp += "Y_"+str(i+1)+r"\\"       
        exp += "\\end{bmatrix} = "+bmatrix(B.reshape(-1,1))
        display(Math(exp))
    for i in range(0,n):
        Y[i] = B[i]
        if (pasos):       
            if (i+1==n): 
                display(Math("\\text{Y por último calculamos } Y_"+str(i+1)))             
            elif (i==0):
                display(Math("\\text{Primero calculamos } Y_"+str(i+1)))                                  
            else:
                display(Math("\\text{Ahora calculamos } Y_"+str(i+1)))                    
            exp = "Y_"+str(i+1)+" = \\frac{B_"+str(i+1)
            expN = "Y_"+str(i+1)+" = \\frac{" + str(B[i])           
        for j in range (0,i):
            Y[i] += -L[i,j]*Y[j]
            if (pasos):
                exp  += " - L_{"+str(i+1)+","+str(j+1)+"} \cdot Y_"+str(j+1) 
                expN += " - "+str(L[i,j])+" \cdot "+str(Y[j])            
        Y[i] = Y[i] / L[i,i]
        if (pasos):
            exp += "}{L_{"+str(i+1)+","+str(i+1)+"}}"
            expN += "}{1} = "+str([i])
            display(Math(exp))
            display(Math(expN))
    if (pasos):
        display(Math("\\text{Y finalmente:}"))                
        display(Math("Y = "+bmatrix(Y)))               
    return Y

In [6]:
Y = sustDirecta(L, B)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Sustitución Inversa:

Utilizaremos la sustitución inversa para hallar el vector $X$ que resuelve nuestro SEL mediante la resolución del sistema $U \cdot X=Y$. Para ello, tenemos el siguiente código, al que se le puede aplicar un tercer parámetro `False` para ahorrar la salida paso a paso:

In [7]:
def sustInversa(U, Y ,pasos = True):
    n = np.shape(U)[0]
    X = np.zeros(n)
    if (pasos):
        display(Math("\\text{Tenemos:}"))                
        exp = bmatrix(U)+" \cdot \\begin{bmatrix}"
        for i in range(0,n): exp += "X_"+str(i+1)+r"\\"       
        exp += "\\end{bmatrix} = "+bmatrix(Y.reshape(-1,1))
        display(Math(exp))  
    for i in reversed(range(0,n)):
        X[i] = Y[i]        
        if (pasos):       
            if (i+1==n): 
                display(Math("\\text{Primero calculamos } X_"+str(i+1)))           
            elif (i==0):
                display(Math("\\text{Y por último calculamos } X_"+str(i+1)))                    
            else:
                display(Math("\\text{Ahora calculamos } X_"+str(i+1)))                    
            exp = "X_"+str(i+1)+" = \\frac{Y_"+str(i+1)
            expN = "X_"+str(i+1)+" = \\frac{" + str(Y[i])               
        for j in range (i+1,n):
            X[i] += -U[i,j]*X[j]
            if (pasos):
                exp  += " - U_{"+str(i+1)+","+str(j+1)+"} \cdot X_"+str(j+1) 
                expN += " - "+str(U[i,j])+" \cdot "+str(X[j])
        X[i] = X[i] / U[i,i]  
        if (pasos):
            exp += "}{U_{"+str(i+1)+","+str(i+1)+"}}"
            expN += "}{"+str(U[i,i])+"} = "+str(X[i])
            display(Math(exp))
            display(Math(expN))    
    if (pasos):
        display(Math("\\text{Y finalmente:}"))                
        display(Math("X = "+bmatrix(X)))   
    return X

In [8]:
Xdoolittle = sustInversa(U, Y)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Ahora podemos realizar algunas verificaciones básicas para chequear el éxito de nuestras operaciones:

Primero que nada... se cumple $ A = L \cdot U $

In [9]:
np.dot(L,U) - A

matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

Y lo más importante, se cumple $ A \cdot X = B $

In [10]:
print(np.dot(A,Xdoolittle))
print(B)

[[1. 2. 3.]]
[1 2 3]


### Factorización de Cholesky:

Esta alternativa se basa en considerar que L y U serán transpuestas entre sí, o sea $A = S \cdot S^T $. Con estas condiciones, las fórmulas se simplifican notablemente:

$$L_{i,i} . U_{i,i} = A_{i,i} - \sum_{k=1}^{i-1}{L_{i,k}.U_{k,i}} \hspace{5mm} con \hspace{5mm} L_{i,i} = U_{i,i} = S_{i,i} \implies
S_{i,i} = \sqrt{A_{i,i} - \sum_{k=1}^{i-1}{S_{i,k}^2}}$$

$$S_{j,i} = \frac{A_{j,i} - \sum_{k=1}^{i-1}{S_{j,k}.S_{i,k}}}{S_{i,i}} \hspace{5mm} con \hspace{5mm} j \in [i+1,n]$$

Como así también el costo computacional, ya que debe calcularse -y almacenarse- solo una matriz $S$ en lugar de $L$ y $U$. Como contrapartida, el método exige que la matriz sea simétrica y definida positiva.

Vamos a crear una función `factorizarCholesky` similar a la vista para Doolittle:

In [11]:
def factorizarCholesky(A,pasos=True):
    n = np.shape(A)[0]
    S = np.zeros((n,n))
    for i in range (0,n):
        S[i,i] = A[i,i]  
        if (pasos):
            display(Math("\\text{Vamos a calcular la "+str(i+1)+"ª  columna de S}"))
            exp = "S_{"+str(i+1)+","+str(i+1)+"} = \sqrt{ A_{"+str(i+1)+","+str(i+1)+"}"
            expN = "S_{"+str(i+1)+","+str(i+1)+"} = \sqrt{"+str(A[i,i])        
        for k in range(0,i):
            S[i,i] += - S[i,k]**2
            if (pasos):
                exp += " - S_{"+str(i+1)+","+str(k+1)+"}^2"
                expN += " - "+str(S[i,k])+"^2"
        S[i,i] = np.sqrt(S[i,i])
        if (pasos):
            exp += "}"            
            expN += "} ="+str(S[i,i])            
            if (i>2):
                display(Math(exp))
                display(Math(expN))
            else:
                display(Math(exp+"\\implies "+expN))       
        for j in range (i+1,n):     
            S[j,i] = A[j,i]
            if (pasos):
                exp = "S_{"+str(j+1)+","+str(i+1)+"} = \\frac{A_{"+str(j+1)+","+str(i+1)+"}" 
                expN = "S_{"+str(j+1)+","+str(i+1)+"} = \\frac{"+str(A[j,i])                
            for k in range(0,i):           
                S[j,i] += - S[j,k]*S[i,k]                  
                if (pasos):
                    exp  += " - S_{"+str(j+1)+","+str(k+1)+"} \cdot S_{"+str(i+1)+","+str(k+1)+"}"  
                    expN += " - "+str(S[j,k])+" \cdot "+ str(S[i,k])                  
            S[j,i] = S[j,i] / S[i,i]
            if (pasos):            
                exp  += "}{S_{"+str(i+1)+","+str(i+1)+"}}"  
                expN += "}{"+str(S[i,i])+"} = "+str(S[j,i])              
                if (i>2):
                    display(Math(exp))
                    display(Math(expN))                
                else:
                    display(Math(exp+"\\implies "+expN))
        if (pasos):
            if (i<n-1):
                display(Math("\\text{Y hasta acá tenemos:}"))                
            else:
                display(Math("\\text{Y ahora sí tenemos S completa:}"))
            display(Math("S = "+bmatrix(S)+" \\hspace{10mm} A = "+bmatrix(A)))
    return S

In [12]:
S = factorizarCholesky(A)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Vamos a verificar si $S$ cumple con la condición esencial de Cholesky:

In [13]:
print(np.dot(S,np.transpose(S)))
print(A)

[[15.  9.  9.]
 [ 9. 12.  7.]
 [ 9.  7.  7.]]
[[15  9  9]
 [ 9 12  7]
 [ 9  7  7]]


Y ahora aplicamos primero la sustitución directa y luego la inversa:

In [14]:
Y = sustDirecta(S, B)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
Xcholesky = sustInversa(S.transpose(), Y)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
B - np.dot(A,Xcholesky)

matrix([[ 1.11022302e-15, -8.88178420e-16,  8.88178420e-16]])

In [17]:
print(Xcholesky, Xdoolittle)

[-0.83333333 -0.2         1.7       ] [-0.83333333 -0.2         1.7       ]
